In [17]:
import glam
import pandas as pd
import numpy as np
import os.path

import matplotlib.pyplot as plt

from scipy.stats import invgauss
import seaborn as sns
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
from scipy import stats
from scipy.stats.stats import pearsonr # Pearson's correlation
import statsmodels.api as sm

#import warnings
#warnings.filterwarnings('ignore')

In [18]:
import rpy2

%load_ext rpy2.ipython
# Set up interface with R
# Make it easy to set and find values in a multi-index DF
idx = pd.IndexSlice
# Set up interface with R
%config InlineBackend.figure_format = 'retina'
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [19]:
data = pd.read_csv(str('data/FF2018_data/GlamDataFF2018_Like_NoBin.csv'))
# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]

In [20]:
# Load data for FF2018 study 
data_all = pd.read_csv('data/FF2018_data/FFAllData_Like.csv')
data_all = data_all.drop(['Unnamed: 0'], axis=1)


In [21]:
data

,subject,trial,choice,rt,item_value_0,item_value_1,gaze_0,gaze_1
0,0,0,0,2009,1.10,0.95,0.568396,0.431604
1,0,1,0,3371,2.00,1.70,0.762332,0.237668
2,0,2,1,1700,1.10,2.30,0.446809,0.553191
3,0,3,1,7466,1.25,1.40,0.532352,0.467648
4,0,4,1,1889,2.00,2.30,0.529736,0.470264
5,0,5,1,2227,1.70,1.85,0.370529,0.629471
6,0,6,0,4259,1.85,0.65,0.482254,0.517746
7,0,7,0,3505,1.55,0.80,0.670739,0.329261
8,0,8,0,2548,1.55,1.10,0.528497,0.471503
9,0,9,1,3226,1.10,2.00,0.581579,0.418421


In [ ]:
%%R
library(lme4)
library(car)
library(optimx)
library(ggplot2)
library(MASS)
library(broom)
library(dplyr)
library(reshape2)
library(arm)
library(multcomp)
library(pbkrtest)